<a href="https://colab.research.google.com/github/ashigupta99/PPOC_ML_Project/blob/main/Final_FakeNews/FakeNewsDetection_ModelScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install dependencies
!pip install transformers datasets accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00


In [2]:
# 2. Imports
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

In [4]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# 3. Load and preprocess data
true_df = pd.read_csv("/content/drive/MyDrive/True.csv")
fake_df = pd.read_csv("/content/drive/MyDrive/Fake.csv")
true_df["label"] = 1
fake_df["label"] = 0
df = pd.concat([true_df, fake_df]).sample(frac=1, random_state=42).reset_index(drop=True)
df["text"] = df["title"].astype(str) + " " + df["text"].astype(str)
df = df[["text", "label"]]

In [7]:
# 4. Split data (80/10/10)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

In [8]:
# 5. Tokenization
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
def tokenize(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=256)

train_encodings = tokenize(train_df["text"].tolist())
val_encodings = tokenize(val_df["text"].tolist())
test_encodings = tokenize(test_df["text"].tolist())

In [9]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_df["label"].tolist())
val_dataset = NewsDataset(val_encodings, val_df["label"].tolist())
test_dataset = NewsDataset(test_encodings, test_df["label"].tolist())

In [10]:
# 6. Model and training setup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    report_to="none"
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-11-119389753.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# 7. Train
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.009600,0.003109
2,0.000100,0.000445


TrainOutput(global_step=8980, training_loss=0.008970643631400403, metrics={'train_runtime': 1455.9746, 'train_samples_per_second': 49.339, 'train_steps_per_second': 6.168, 'total_flos': 9450422886420480.0, 'train_loss': 0.008970643631400403, 'epoch': 2.0})

In [21]:
# 8. Temperature Scaling (on validation set)
import torch
import torch.nn as nn
import torch.optim as optim
import pickle

# Collect logits and labels from validation set
val_logits = []
val_labels = []
model.eval()
for batch in val_dataset:
    inputs = {k: v.unsqueeze(0).to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
        val_logits.append(outputs.logits.cpu())
        val_labels.append(batch['labels'].item())
val_logits = torch.cat(val_logits, dim=0)
val_labels = torch.tensor(val_labels)

# Temperature scaling wrapper
class ModelWithTemperature(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)  # Start >1

    def forward(self, **inputs):
        logits = self.model(**inputs).logits
        return logits / self.temperature

# Optimize temperature
model_temp = ModelWithTemperature(model)
optimizer = optim.LBFGS([model_temp.temperature], lr=0.01, max_iter=50)
def eval():
    optimizer.zero_grad()
    logits = val_logits / model_temp.temperature
    loss = nn.CrossEntropyLoss()(logits, val_labels)
    loss.backward()
    return loss
optimizer.step(eval)
print(f"Optimal temperature: {model_temp.temperature.item():.3f}")

# Save the temperature value for inference
with open("temperature.pkl", "wb") as f:
    pickle.dump(model_temp.temperature.item(), f)

Optimal temperature: 1.499


In [22]:
# 8. Evaluate
metrics = trainer.evaluate(test_dataset)
print("\nTest set metrics:", metrics)
preds = trainer.predict(test_dataset)
y_true = preds.label_ids
y_pred = preds.predictions.argmax(axis=1)
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=["Fake", "Real"]))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))



Test set metrics: {'eval_loss': 0.005283174104988575, 'eval_runtime': 17.8198, 'eval_samples_per_second': 251.967, 'eval_steps_per_second': 31.538, 'epoch': 2.0}

Classification Report:
               precision    recall  f1-score   support

        Fake       1.00      1.00      1.00      2348
        Real       1.00      1.00      1.00      2142

    accuracy                           1.00      4490
   macro avg       1.00      1.00      1.00      4490
weighted avg       1.00      1.00      1.00      4490

Confusion Matrix:
 [[2346    2]
 [   0 2142]]


In [23]:
# 9. Save model and tokenizer for Streamlit
model.save_pretrained("saved_model/")
tokenizer.save_pretrained("saved_model/")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [25]:
import torch
import pickle

# Load model, tokenizer, and optimal temperature
from transformers import BertForSequenceClassification, BertTokenizerFast

model = BertForSequenceClassification.from_pretrained("saved_model/")
tokenizer = BertTokenizerFast.from_pretrained("saved_model/")
with open("temperature.pkl", "rb") as f:
    optimal_temp = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_with_temperature(text, model, tokenizer, temperature):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
        logits = logits / temperature
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1)
        confidence = probs[0, pred.item()].item()
    label = "Real" if pred.item() == 1 else "Fake"
    return label, confidence

# Example: Predict on new headlines
headlines = [
    "NASA’s Perseverance rover begins mission on Mars to collect rock samples.",
    "COVID-19 vaccine causes magnetic arms in humans.",
    "India becomes the first country to land on Moon’s south pole.",
    "Bill Gates admits to creating coronavirus.",
]

for text in headlines:
    label, conf = predict_with_temperature(text, model, tokenizer, optimal_temp)
    print(f"{text}\n → {label} (Confidence: {conf*100:.1f}%)\n")


NASA’s Perseverance rover begins mission on Mars to collect rock samples.
 → Real (Confidence: 96.4%)

COVID-19 vaccine causes magnetic arms in humans.
 → Fake (Confidence: 99.9%)

India becomes the first country to land on Moon’s south pole.
 → Real (Confidence: 98.3%)

Bill Gates admits to creating coronavirus.
 → Fake (Confidence: 99.9%)

